In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,881 kB]
Hit:12 http://ppa.launc

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-18 00:49:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-18 00:49:17 (6.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:

# spark = SparkSession \
#     .builder \
#     .appName("Python Spark SQL basic example") \
#     .config("spark.jars", "/path_to_postgresDriver/postgresql-42.2.5.jar") \
#     .getOrCreate()

vine_review_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://fornpl.cd1z6z2mod7a.us-east-2.rds.amazonaws.com:5432/awsChanllenge") \
    .option("dbtable", "vine_table") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

vine_review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [ ]:
filered_df = vine_review_df.filter((vine_review_df['total_votes'] >20) & (vine_review_df['helpful_votes'] >0))

In [ ]:
filered_df1 = filered_df.filter(filered_df['helpful_votes']/filered_df['total_votes'] >= 0.5)

In [ ]:
vine_pg_df = filered_df1.filter(filered_df1['vine'] =='Y')

In [ ]:
unvine_pg_df = filered_df1.filter(filered_df1['vine'] =='N')

In [ ]:
filered_df1.groupby(['vine']).count().show()

+----+------+
|vine| count|
+----+------+
|   N|378638|
+----+------+



In [ ]:
total_view = vine_review_df.count()
print(total_view)
five_star = vine_review_df.filter(vine_review_df.star_rating == 5).count()
print(five_star)
vine_five = vine_review_df.filter((vine_review_df.vine == 'Y') & (vine_review_df.star_rating == 5) ).count()/five_star
non_vine_five = vine_review_df.filter((vine_review_df.vine == 'N') & (vine_review_df.star_rating == 5)).count()/five_star
print(vine_five)
print(non_vine_five)


3105520
1864804
5.36249385994453e-07
0.999999463750614


In [ ]:
print(vine_review_df.filter(vine_review_df.vine == 'Y').count())
print(vine_review_df.filter(vine_review_df.vine == 'N').count())

2
3105513


In [ ]:
print(vine_review_df.filter(vine_review_df.vine == 'Y').count())
print(vine_review_df.filter(vine_review_df.vine == 'N').count())

In [ ]:
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     